<a href="https://colab.research.google.com/github/ahmed-sala/NLP-Assignment/blob/main/20210064_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping


In [23]:
data = pd.read_csv('/content/drive/MyDrive/IMDB Dataset.csv')

texts = data['label'].astype(str)
labels = data['sentiment']

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

max_words = 10000
max_len = 200

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

X = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')

X_train, X_test, y_train, y_test = train_test_split(
    X, labels_encoded, test_size=0.2, random_state=42
)



In [30]:
model = Sequential([
    Embedding(max_words, 128, input_length=max_len),
    Bidirectional(LSTM(128)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


history = model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=64,
    validation_split=0.1,
)



Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


563/563 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - accuracy: 0.6884 - loss: 0.5639 - val_accuracy: 0.8635 - val_loss: 0.3644
Epoch 2/5
563/563 ━━━━━━━━━━━━━━━━━━━━ 21s 25ms/step - accuracy: 0.8822 - loss: 0.2975 - val_accuracy: 0.8808 - val_loss: 0.3016
Epoch 3/5
563/563 ━━━━━━━━━━━━━━━━━━━━ 20s 24ms/step - accuracy: 0.9152 - loss: 0.2246 - val_accuracy: 0.8745 - val_loss: 0.3173
Epoch 4/5
563/563 ━━━━━━━━━━━━━━━━━━━━ 20s 24ms/step - accuracy: 0.9466 - loss: 0.1516 - val_accuracy: 0.8630 - val_loss: 0.3567
Epoch 5/5
563/563 ━━━━━━━━━━━━━━━━━━━━ 21s 24ms/step - accuracy: 0.9595 - loss: 0.1147 - val_accuracy: 0.8472 - val_loss: 0.4035


In [31]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

y_pred = (model.predict(X_test) > 0.5).astype("int32")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8520 - loss: 0.4084
Test Accuracy: 85.13%
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step

Classification Report:
              precision    recall  f1-score   support

    negative       0.89      0.80      0.84      4961
    positive       0.82      0.91      0.86      5039

    accuracy                           0.85     10000
   macro avg       0.86      0.85      0.85     10000
weighted avg       0.86      0.85      0.85     10000



In [32]:
predictions = model.predict(X_test[:5])

print("\nPredictions for the first 5 test samples:")
for pred in predictions:
    label = "positive" if pred[0] >= 0.5 else "negative"
    print(label)

print("\nTrue values for the first 5 test samples:")
for true_val in y_test[:5]:
    label = "positive" if true_val == 1 else "negative"
    print(label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step

Predictions for the first 5 test samples:
positive
positive
negative
positive
negative

True values for the first 5 test samples:
positive
positive
negative
positive
negative
